# Learning-to-rank : pyTerrier - OpenNIR

Dans cette partie, on s'intéresse à l'utilisation de modèles neuronaux pour la recherche d'information. 
Les modèles neuronaux utilisés ont été rassemblés dans la librairie [OpenNIR](https://opennir.net/). 
On explorera également le modèle T5 avec le plugin [monoT5](https://github.com/terrierteam/pyterrier_t5).

In [1]:
!pip install --upgrade python-terrier
!pip install --upgrade git+https://github.com/Georgetown-IR-Lab/OpenNIR
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5

     |████████████████████████████████| 97 kB 2.4 MB/s 
     |████████████████████████████████| 1.1 MB 30.0 MB/s 
     |████████████████████████████████| 69 kB 2.4 MB/s 
     |████████████████████████████████| 255 kB 39.4 MB/s 
     |████████████████████████████████| 41 kB 106 kB/s 
     |████████████████████████████████| 72 kB 1.2 MB/s 
     |████████████████████████████████| 126 kB 38.3 MB/s 
     |████████████████████████████████| 1.2 MB 38.7 MB/s 
     |████████████████████████████████| 6.4 MB 33.0 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 294 kB 45.4 MB/s 
     |████████████████████████████████| 291 kB 48.6 MB/s 
     |████████████████████████████████| 59 kB 6.6 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.0-py3-none-any.whl size=103883 sha256=f1da85ad3561d6c2d1f20432c232f7c74b541b7727e053f7ef817ec3eef3c00c
  Stored in directory: /root/.cac

## Initialisation
De façon similaire au TP1, on initialise PyTerrier. Nous allons travailler sur le dataset CORD19. le bloc suivant est une répétition du code en TP1.

In [2]:
import pyterrier as pt
if not pt.started():
    pt.init(tqdm='notebook')
import onir_pt

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
import os

dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # création de l'index
    indexer = pt.index.IterDictIndexer(pt_index_path)
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # dans le cas où l'index est déjà créé
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)

[INFO] [starting] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/topics-rnd5.xml: s] [18.7kB] [10.0MB/s]
/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:433: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df.drop(df.columns.difference(['qid','query']), 1, inplace=True)
[INFO] [starting] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt
[INFO] [finished] https://ir.nist.gov/covidSubmit/data/qrels-covid_d5_j0.5-5.txt: [11.16s] [1.14MB] [102kB/s]
[INFO] [starting] building docstore
[INFO] If you have a local copy of https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.csv, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/80d664e496b8b7e50a39c6f6bb92e0ef
[INFO] [starting] https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-07-16/metadata.cs

cord19/trec-covid documents:   0%|          | 0/192509 s<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  from ipykernel import kernelapp as app


15:48:55.290 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (6iu1dtyl) - further warnings are suppressed
15:50:15.853 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 54937 empty documents
15:50:16.059 [ForkJoinPool-1-worker-3] ERROR org.terrier.structures.indexing.Indexer - Could not finish MetaIndexBuilder: 
java.io.IOException: Key 8lqzfj2e is not unique: 37597,11755
For MetaIndex, to suppress, set metaindex.compressed.reverse.allow.duplicates=true
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.mergeTwo(FSOrderedMapFile.java:1374)
	at org.terrier.structures.collections.FSOrderedMapFile$MultiFSOMapWriter.close(FSOrderedMapFile.java:1308)
	at org.terrier.structures.indexing.BaseMetaIndexBuilder.close(BaseMetaIndexBuilder.java:321)
	at org.terrier.structures.indexing.classical.BasicIndexer.createDirectIndex(BasicIndexer.java:346)
	at org.terrier.structures.indexing

In [4]:
print(os.path.exists("terrier_index.zip"))

False


In [5]:
#Si le chargement de la collection est trop long à cause du débit ou autres raisons,
# il est possible de récupérer directement l'index fourni par Terrier
import os

if not os.path.exists("terrier_index.zip"):
  !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
  !unzip -j terrier_index.zip -d terrier_index

index_ref = pt.IndexRef.of("./terrier_index/data.properties")
index = pt.IndexFactory.of(index_ref)

--2022-03-10 15:50:24--  http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
Resolving www.dcs.gla.ac.uk (www.dcs.gla.ac.uk)... 130.209.240.1
Connecting to www.dcs.gla.ac.uk (www.dcs.gla.ac.uk)|130.209.240.1|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42017186 (40M) [application/zip]
Saving to: ‘terrier_index.zip’

terrier_index.zip   100%[===================>]  40.07M  18.2MB/s    in 2.2s    

2022-03-10 15:50:27 (18.2 MB/s) - ‘terrier_index.zip’ saved [42017186/42017186]

Archive:  terrier_index.zip
  inflating: terrier_index/data.lexicon.fsomapfile  
  inflating: terrier_index/data.properties  
  inflating: terrier_index/data.lexicon.fsomaphash  
  inflating: terrier_index/data.lexicon.fsomapid  
  inflating: terrier_index/data.meta.idx  
  inflating: terrier_index/data.direct.bf  
  inflating: terrier_index/data.inverted.bf  
  inflating: terrier_index/data.meta.zdata  
  inflating: terrier_index/data.document.fsarrayfile  


## Modèles de ré-ordonancement neuronaux "from scratch"

Les modèles de ré-ordonnancement dans OpenNIR sont constitués de deux éléments :
*  ranker: un modèle d'ordonnancement (e.g., drmm, knrm, pacrr, ...). Cf la liste des [Rankers](https://opennir.net/rankers.html).
*  vocab : défnit comment le texte est encodé par le modèle (e.g., wordvec_hash, bert, ...). Cela permet ainsi de tester plusieurs méthodes de représentation. Plus de détails concernant le [vocab](https://opennir.net/vocab.html).

Les modèles de réordonnancement s'appuient sur une première étape d'ordonnancement (souvent BM25), récupèrent ensuite les textes des top documents et appliquent ensuite le modèle neuronal.



In [ ]:
knrm = onir_pt.reranker('knrm', 'wordvec_hash', text_field='abstract')

config file not found: config
[2022-03-10 15:50:28,473][WordvecHashVocab][DEBUG] [starting] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip


[2022-03-10 15:50:39,833][onir.util.download][WARNING] no hash provided for https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip; consider adding expected_md5="3cc8839ac3fa9a6187149b1e73328b2a" to ensure data integrity.
[2022-03-10 15:50:39,847][onir.util.download][DEBUG] downloaded https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [10.97s] [682M] [67.0MB/s]
[2022-03-10 15:50:39,860][WordvecHashVocab][DEBUG] [finished] downloading https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip [11.39s]
[2022-03-10 15:50:39,860][WordvecHashVocab][DEBUG] [starting] extracting vecs
[2022-03-10 15:50:58,534][WordvecHashVocab][DEBUG] [finished] extracting vecs [18.67s]
[2022-03-10 15:50:58,534][WordvecHashVocab][DEBUG] [starting] loading vecs into memory


Une fois le modèle chargé, il est nécessaire de mettre en place la pipeline d'évaluation vue dans le tp1. 
Le modèle neuronal n'est pas efficace car il n'a pas été entraîné et utilise des poids aléatoires.

In [ ]:
br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

## Entraînement des modèles sur les jeux de données

Pour entraîner les modèles, il est nécessaire de construire la pipeline de modèles/transformations à utliser et ensuite d'appliquer la fonction .fit() à la pipeline. 
Le code ci-dessous prend beaucoup de temps. Il est donné à titre indicatif si vous souhaitez l'utiliser sur des serveurs adaptés. L'étape suivante permet de charger directement les poids du modèle pré-entraîné à l'avance et mis à disposition de la communauté.

Dans ce qui suit, on utilise le jeu de données MS MARCO medical pour pré-entraîner le modèle qui sera ensuite appliqué sur CORD19. 
Il est également possible de garder seulement le jeu de données CORD19, de le découper en train/val/test et regarder les performances. 

In [ ]:
# Apprentissage du modèle sur des données médicales (MS MARCO medical)
from sklearn.model_selection import train_test_split
train_ds = pt.datasets.get_dataset('irds:msmarco-passage/train/medical')
train_topics, valid_topics = train_test_split(train_ds.get_topics(), test_size=50, random_state=42)

# Indexation de MS MARCO pour la première étape d'ordonnancement et récupérer les textes (pour le ranker openNIR)
indexer = pt.index.IterDictIndexer('./terrier_msmarco-passage')
tr_index_ref = indexer.index(train_ds.get_corpus_iter(), fields=('text',), meta=('docno',))

pipeline = (pt.BatchRetrieve(tr_index_ref) % 100 # récupère les 100 premiers documents
            >> pt.text.get_text(train_ds, 'text') # récupère le texte de ces documents
            >> pt.apply.generic(lambda df: df.rename(columns={'text': 'abstract'})) # renomme la colonne
            >> knrm) # applique le re-ranker


# pipeline.fit(
#     train_topics,
#     train_ds.get_qrels(),
#     valid_topics,
#     train_ds.get_qrels())

Pour éviter l'étape d'entraînement ici, on utlise une version pré-entraînée.

**Important** : penser à supprimer la mémoire des re-rankers qui vont être mis à jour avec les modèles pré-entrainés.

In [ ]:
del knrm # free up the memory before loading a new version of the ranker
knrm = onir_pt.reranker.from_checkpoint('https://macavaney.us/knrm.medmarco.tar.gz', text_field='abstract', expected_md5="d70b1d4f899690dae51161537e69ed5a")

Il est ensuire possible de lancer la pipeline d'évaluation avec ce nouveau modèle. Les résultats sont meilleurs !

In [ ]:
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> knrm
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    baseline=0,       ## spécifie quelle est le modèle de référence pour calculer les améliorations.
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

**Exercice 1**

On souhaite savoir quel est le nombre de documents pertinents qu'il est nécessaire de retourner à l'étape de pré-ordonnancement pour utiliser ensuite le modèle KNRM. Ecrire la pipeline de traitement permettant de trouver la valeur optimale de ce paramètre (valeurs de 10 à 100 par pas de 10).

Remarque : Ne pas faire de train/test exceptionnellement, on souhaite juste constater la valeur qui maximise sur le jeu de données.


In [ ]:
cutoffs = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
dph = pt.BatchRetrieve(index)
res = pt.Experiment(
    [dph % cutoff >> pt.text.get_text(dataset, 'abstract') >> knrm for cutoff in cutoffs],
    dataset.get_topics('description'),
    dataset.get_qrels(),
    names=[f'c={cutoff}' for cutoff in cutoffs],
    eval_metrics=["map", "recip_rank", "ndcg", "ndcg_cut.10", "mrt"]
)
res

## Modèle Vanilla BERT

**Exercice 2**

Sur le même principe que le modèle KNRM, analyser les performances du modèle vanilla BERT sans et avec pré-entraînement.
Pour la version du modèle pré-entraîné, on utilisera le checkpoint ['https://macavaney.us/scibert-medmarco.tar.gz']('https://macavaney.us/scibert-medmarco.tar.gz') avec le paramètre expected_md5="854966d0b61543ffffa44cea627ab63b".

Synthétisez toutes les mesures d'évaluation dans un même tableau (bm25, knrm et Vanilla Bert / avec/sans entraînement).

In [ ]:
## VBert sans pré-traitement
vbert = onir_pt.reranker('vbert', 'wordvec_hash', text_field='abstract')

br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> vbert
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

In [ ]:
## Chargement du checkpoint
del vbert
vbert = onir_pt.reranker.from_checkpoint('https://macavaney.us/scibert-medmarco.tar.gz', text_field='abstract', expected_md5="854966d0b61543ffffa44cea627ab63b")

br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> vbert
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> KNRM'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

In [ ]:
## Comparaison des performances

# MonoT5

Pour expérimenter le modèle T5, on utilisera une librairie différente implémentée par pyTerrier. 

In [ ]:
from pyterrier_t5 import MonoT5ReRanker
monoT5 = MonoT5ReRanker(text_field='abstract')

br = pt.BatchRetrieve(index) % 30
pipeline = (br >> pt.text.get_text(dataset, 'abstract') >> monoT5)
pt.Experiment(
    [br, pipeline],
    dataset.get_topics('description'),
    dataset.get_qrels(),
    names=['DPH', 'DPH >> T5'],
    eval_metrics=["map", "recip_rank", "ndcg", "ndcg_cut.10", "mrt"]
)